<a href="https://colab.research.google.com/github/danlingzhou16/stat390/blob/Danling/Danling_covid_project_imputation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
## libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, train_test_split, cross_val_predict
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression

import time as time

In [3]:
data = pd.read_csv('/content/top_10_gdp_covid_data_imputed-2.csv')
# Grace revised the dataset and it is downloaded as to_10_gdp_covid_data_imputed-2.csv

<ipython-input-3-c1a624b368d9>:1: DtypeWarning: Columns (168) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/top_10_gdp_covid_data_imputed-2.csv')


In [4]:
data.head()

,Unnamed: 0.1,Unnamed: 0,new_deceased,population,population_male,population_female,latitude,longitude,area_sq_km,life_expectancy,...,place_id,wikidata_id,country_code,country_name,subregion1_code,subregion1_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,location_key_y,location_key
0,0,9896188.0,0.0,733391.0,424916.0,391925.0,64.0,-150.0,1717856.0,78.0,...,ChIJG8CuwJzfAFQRNduKqSde27w,Q797,US,United States of America,AK,Alaska,US,USA,US,US
1,1,9896189.0,0.0,733391.0,424916.0,391925.0,64.0,-150.0,1717856.0,78.0,...,ChIJG8CuwJzfAFQRNduKqSde27w,Q797,US,United States of America,AK,Alaska,US,USA,US,US
2,2,9896190.0,0.0,733391.0,424916.0,391925.0,64.0,-150.0,1717856.0,78.0,...,ChIJG8CuwJzfAFQRNduKqSde27w,Q797,US,United States of America,AK,Alaska,US,USA,US,US
3,3,9896191.0,0.0,733391.0,424916.0,391925.0,64.0,-150.0,1717856.0,78.0,...,ChIJG8CuwJzfAFQRNduKqSde27w,Q797,US,United States of America,AK,Alaska,US,USA,US,US
4,4,9896192.0,0.0,733391.0,424916.0,391925.0,64.0,-150.0,1717856.0,78.0,...,ChIJG8CuwJzfAFQRNduKqSde27w,Q797,US,United States of America,AK,Alaska,US,USA,US,US


In [7]:
for col in data.columns:
  print(col)

Unnamed: 0.1
Unnamed: 0
new_deceased
population
population_male
population_female
latitude
longitude
area_sq_km
life_expectancy
mobility_workplaces
aggregation_level
gdp_usd
gdp_per_capita_usd
AG.LND.AGRI.K2
AG.LND.AGRI.ZS
AG.LND.ARBL.HA
AG.LND.ARBL.HA.PC
AG.LND.ARBL.ZS
AG.LND.CROP.ZS
AG.LND.FRST.K2
AG.LND.FRST.ZS
AG.LND.TOTL.K2
AG.PRD.CROP.XD
AG.PRD.FOOD.XD
AG.PRD.LVSK.XD
AG.SRF.TOTL.K2
BX.KLT.DINV.CD.WD
EG.ELC.ACCS.RU.ZS
EG.ELC.ACCS.UR.ZS
EG.ELC.ACCS.ZS
EG.ELC.RNEW.ZS
EG.FEC.RNEW.ZS
EN.ATM.CO2E.GF.KT
EN.ATM.CO2E.GF.ZS
EN.ATM.CO2E.KT
EN.ATM.CO2E.LF.KT
EN.ATM.CO2E.LF.ZS
EN.ATM.CO2E.PC
EN.ATM.CO2E.SF.KT
EN.ATM.CO2E.SF.ZS
EN.ATM.PM25.MC.M3
EN.ATM.PM25.MC.T1.ZS
EN.ATM.PM25.MC.T2.ZS
EN.ATM.PM25.MC.T3.ZS
EN.ATM.PM25.MC.ZS
EN.BIR.THRD.NO
EN.FSH.THRD.NO
EN.HPT.THRD.NO
EN.MAM.THRD.NO
EN.POP.DNST
ER.FSH.CAPT.MT
ER.FSH.PROD.MT
ER.LND.PTLD.ZS
ER.PTD.TOTL.ZS
IP.JRN.ARTC.SC
IT.CEL.SETS
IT.CEL.SETS.P2
IT.MLT.MAIN
IT.MLT.MAIN.P2
IT.NET.BBND
IT.NET.BBND.P2
IT.NET.SECR
IT.NET.SECR.P6
IT.NET.USER.ZS
NV.

In [89]:
# remove the columns that are unlikely to become useful
#aggregation level indicates how the dataset is put together 0 = country; 1 = state/province level
not_useful_col = ['Unnamed: 0.1','Unnamed: 0', 'aggregation_level','location_key', 'location_key_y', 'place_id', 'wikidata_id', 'subregion1_code', 'iso_3166_1_alpha_2', 'iso_3166_1_alpha_3', 'subregion1_name']

In [90]:
data_removed_col = data.drop(columns = not_useful_col)

In [91]:
data_not_imputed = pd.read_csv('top_10_gdp_covid_data.csv')

<ipython-input-91-416d100947f4>:1: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  data_not_imputed = pd.read_csv('top_10_gdp_covid_data.csv')


In [92]:
vacc_and_govt_policy = data_not_imputed[['new_persons_fully_vaccinated','cumulative_persons_fully_vaccinated', 'school_closing','workplace_closing',
                                         'cancel_public_events','restrictions_on_gatherings','public_transport_closing','stay_at_home_requirements','restrictions_on_internal_movement',
                                         'international_travel_controls','income_support','debt_relief','public_information_campaigns','contact_tracing','testing_policy','facial_coverings','vaccination_policy']]

In [93]:
vacc_and_govt_policy['new_persons_fully_vaccinated'] = vacc_and_govt_policy['new_persons_fully_vaccinated'].fillna(0)

<ipython-input-93-bfb029ffcb01>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacc_and_govt_policy['new_persons_fully_vaccinated'] = vacc_and_govt_policy['new_persons_fully_vaccinated'].fillna(0)


In [94]:
data_complete = pd.concat([data_removed_col,vacc_and_govt_policy], axis = 1)

In [95]:
data_complete

,new_deceased,population,population_male,population_female,latitude,longitude,area_sq_km,life_expectancy,mobility_workplaces,gdp_usd,...,stay_at_home_requirements,restrictions_on_internal_movement,international_travel_controls,income_support,debt_relief,public_information_campaigns,contact_tracing,testing_policy,facial_coverings,vaccination_policy
0,0.0,733391.0,424916.0,391925.0,64.0,-150.0,1717856.0,78.0,-10.333333,2.137442e+13,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
1,0.0,733391.0,424916.0,391925.0,64.0,-150.0,1717856.0,78.0,-10.333333,2.137442e+13,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
2,0.0,733391.0,424916.0,391925.0,64.0,-150.0,1717856.0,78.0,-10.333333,2.137442e+13,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
3,0.0,733391.0,424916.0,391925.0,64.0,-150.0,1717856.0,78.0,-10.333333,2.137442e+13,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
4,0.0,733391.0,424916.0,391925.0,64.0,-150.0,1717856.0,78.0,-10.333333,2.137442e+13,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97179,0.0,2656156.0,1120500.0,1111402.0,-26.0,121.0,2527013.0,81.0,-73.333333,1.392681e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97180,0.0,2656156.0,1120500.0,1111402.0,-26.0,121.0,2527013.0,81.0,-73.333333,1.392681e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97181,0.0,2656156.0,1120500.0,1111402.0,-26.0,121.0,2527013.0,81.0,-73.333333,1.392681e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97182,0.0,2656156.0,1120500.0,1111402.0,-26.0,121.0,2527013.0,81.0,-73.333333,1.392681e+12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
solve_cumulative = data_complete[['location_key_x','date', 'new_persons_fully_vaccinated', 'cumulative_persons_fully_vaccinated']]

In [97]:
solve_cumulative_grouped = solve_cumulative.groupby(['location_key_x'])

In [98]:
cumulative_persons_fully_vaccinated_filledna = solve_cumulative_grouped['cumulative_persons_fully_vaccinated'].fillna(method = 'ffill')
cumulative_persons_fully_vaccinated_filledna = cumulative_persons_fully_vaccinated_filledna.fillna(0)

In [99]:
data_complete.cumulative_persons_fully_vaccinated = cumulative_persons_fully_vaccinated_filledna

In [100]:
print(data_complete.loc[:,data_complete.isnull().any(axis=0)])

       school_closing  workplace_closing  cancel_public_events  \
0                 0.0                0.0                   0.0   
1                 0.0                0.0                   0.0   
2                 0.0                0.0                   0.0   
3                 0.0                0.0                   0.0   
4                 0.0                0.0                   0.0   
...               ...                ...                   ...   
97179             NaN                NaN                   NaN   
97180             NaN                NaN                   NaN   
97181             NaN                NaN                   NaN   
97182             NaN                NaN                   NaN   
97183             NaN                NaN                   NaN   

       restrictions_on_gatherings  public_transport_closing  \
0                             0.0                       0.0   
1                             0.0                       0.0   
2                 

In [101]:
# just fill the na with 0. if there is no information about the policy, it is likely that there is no such policy
data_complete = data_complete.fillna(0)

In [103]:
data_complete.to_csv('complete_top_10_imputation.csv')